<font size=50 color=darkblue>Shortest Path ILP</font>

# Problem modelling with DOcplex

## Import necessary modules

- DOcplex will be used to model and solve the Shortest-Path ILP
- Import module `itertools` to group links based on their tails/heads

In [ ]:
from docplex.mp.model import Model
import itertools

## Shortest-Path ILP <font size=3>(variables are colored blue)</font>
**Minimize**
### $$\sum_{(u,v)\in \mathcal{E}}c_{(u,v)}\cdot \color{blue}x_{(u,v)}$$
**Subject to**
### \begin{align*}
{\color{blue}x}_{({\color{red}t},{\color{green}s})} = 1\,\qquad&\\
\sum_{(u,v)\in{\color{red}\delta}^{\color{red}-}_{(v)}}{\color{blue}x_{(u,v)}} - \sum_{(v,u)\in{\color{green}\delta}^{\color{green}+}_{(v)}}{\color{blue}x_{(v,u)}} = 0,\qquad&\forall v\in \mathcal{V}\\
{\color{blue}x_{(u,v)}}\in\left\{0,1\right\},\qquad&\forall(u,v)\in \mathcal{E}
\end{align*}

## Display the network
<img src='img/sp.png' width=700/>

## Define the model input
### \begin{align*}
\mathcal{V} =\,& \{0, 1, 2, 3, 4, 5, 6\}&\\
\mathcal{E} =\,& \{(0,1), (0,2), (1,3), (1,4), (2,3), (2,5), (3,4), (3,5), (3,6), (4,6), (5,6)\}&\\
{\color{green}s} =\,& 0&\\
{\color{red}t} =\,& 6&
\end{align*}

In [ ]:
V = [0, 1, 2, 3, 4, 5, 6]
E = [(0,1), (0,2), (1,3), (1,4), (2,3), (2,5), (3,4), (3,5), (3,6), (4,6), (5,6)]
s, t = 0, 6

## Augment the graph with the imaginary link (<font color='red'>t</font>, <font color='green'>s</font>)
### $$\mathcal{E}:=\mathcal{E}\cup \{({\color{red}t},{\color{green}s})\}$$

In [ ]:
E.append((t,s))

## Define in-links <code><font color='red'>delta_min</font></code> and out-links <code><font color='green'>delta_pls</font></code>, subscriptable by node indices
### \begin{align*}
{\color{red}\delta}^{\color{red}-}_{(v)} = \{e\in \mathcal{E}\,|&\,\text{head}(e)\equiv v \},&\quad\forall v\in \mathcal{V}\\
{\color{green}\delta}^{\color{green}+}_{(v)} = \{e\in \mathcal{E}\,|&\,\text{tail}(e)\equiv v \},&\quad\forall v\in \mathcal{V}\\
\end{align*}

In [ ]:
# Anonymous functions
head = lambda e: e[1]
tail = lambda e: e[0]
delta_min = {v: set(delt_min) for v, delt_min in itertools.groupby(sorted(E, key=head), head)}
delta_pls = {v: set(delt_pls) for v, delt_pls in itertools.groupby(sorted(E, key=tail), tail)}

## Define link costs $c_{(e)},\,\forall e\in \mathcal{E}$, subscriptable by link indices <font size=3>
### (Note that the cost of the imaginary link (<font color=red>t</font>,<font color=green>s</font>) is 0)</font>

In [ ]:
c = dict(zip(E, [15, 20, 10, 25, 15, 20, 20, 15, 30, 10, 20, 0]))

## Create the ILP model for Shortest-Path problem using DOcplex

In [ ]:
sp_ILP = Model(name='Shortest Path')

## Define decision variables as binaries: ${\color{blue}x_{(e)}}\in \{0,1\},\quad\forall e\in \mathcal{E}$

In [ ]:
sp_ILP.clear() # This line is optional. Its purpose is to avoid variable duplicates
x = {(u,v): sp_ILP.binary_var(name=f'x({u},{v})') for u,v in E}

## Define demand satisfaction constraint $${\color{blue}x}_{({\color{red}t},{\color{green}s})} = 1\qquad$$

In [ ]:
demand_sat_ct = sp_ILP.add_constraint(ct=x[t,s]==1, ctname=f'demand_satisfaction')

## Define flow conservation constraints, subscriptable by node indices $$\sum_{(u,v)\in{\color{red}\delta}^{{\color{red}-}}_{(v)}}{\color{blue}x_{(u,v)}} - \sum_{(v,u)\in{\color{green}\delta}^{{\color{green}+}}_{(v)}}{\color{blue}x_{(v,u)}} = 0,\quad\forall v\in \mathcal{V}$$

In [ ]:
flow_conserv_ct = {v: sp_ILP.add_constraint(ct=sp_ILP.sum(x[e] for e in delta_min[v]) - sp_ILP.sum(x[e] for e in delta_pls[v]) == 0,
                                            ctname=f'flow_conservation({v})') for v in V}

## Define objective function $$\textbf{minimize}\qquad \sum_{(u,v)\in \mathcal{E}}c_{(u,v)}\cdot \color{blue}x_{(u,v)}$$

In [ ]:
sp_ILP.minimize(sp_ILP.sum(c[e]*x[e] for e in E))

## Summarize the model

In [ ]:
sp_ILP.print_information()

## Solve the ILP and display the result

In [ ]:
sp_sol = sp_ILP.solve()
if sp_sol:
    sp_sol.display()

# Result Visualization

## Import visualization modules
- `igraph`, `matplotlib`

In [ ]:
import igraph as ig
import matplotlib.pyplot as plt

## For convenience, extract the solution to a dictionary named `sol_x`

In [ ]:
sol_x = sp_sol.get_value_dict(x)
sol_x

## Since the link (<font color='red'>t</font>, <font color='green'>s</font>) is imaginary, remove it from the link list `E`

In [ ]:
E.remove((t,s))

## Instantiate a `Graph` object with module `igraph`
### Notes
- __*Node(s)*__ is/are called __*vertex/vertices*__ in `igraph`
- __*Link(s)*__ is/are called __*edge/edges*__ in `igraph`
- The edge list is sufficient to instantiate a `Graph` object. The vertex list is automatically inferred by `igraph` (based on the tails/heads' indices).

In [ ]:
g = ig.Graph(edges=E, directed=True)

## Visualize the graph

In [ ]:
g.vs['label'] = g.vs.indices
g.vs['size'] = 50
g.vs['color'] = 'white'
g.vs[s,t]['color'] = 'green', 'red'

lnk_flow_max = max(sol_x.values())
g.es['label'] = [c[e] for e in E]
g.es['arrow_size'] = [10 if sol_x[e] > 0 else 6 for e in E]
g.es['arrow_width'] = [10 if sol_x[e] > 0 else None for e in E]
g.es['width'] = [5 if sol_x[e] > 0 else 0.5 for e in E]
g.es['color'] = ['darkblue' if sol_x[e] > 0 else None for e in E]
g.es['label_size'] = 15

fig, ax = plt.subplots()
fig.set_size_inches(10,10)

p = ig.plot(g, layout=g.layout('kk'), edge_label=g.es['label'], edge_background='white', target=ax)
    
plt.show()